# Predicció de notes per fer recomanació de matrícules

El nostre objectiu d'aquest notebook és fer un ranking de notes basats amb un algoritme de predicció. Tenim un conjunt de dades normalitzades i unes altres no normalitzades, totes les proves les farem amb els dos conjunts. Hem creat un recomanador propi i l'hem integrat amb l'interfaç d'sklearn per així poder testejar de la mateixa manera que fem amb altres algoritmes d'aquesta llibreria. 

In [1]:
%matplotlib inline
from IPython.display import display
import numpy as np
import pandas as pd
import math
import random as rnd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

#import from utils
import sys
sys.path.append('../')
from utils import *

reg = pd.read_csv("../../../registers.csv", na_values=np.nan)
assig = pd.read_csv("../../../assigs.csv", na_values=np.nan)
conv = pd.read_csv("../../../conv.csv", na_values=np.nan)
qual = pd.read_csv("../../../qualifications.csv", na_values=np.nan)
qual_norm = pd.read_csv("../../../qualifications_normalized.csv", na_values=np.nan)

reg_info = reg[reg['id_enseny'] != 'G1042']
assig_info = assig[assig['id_enseny_assig'] != 'G1042']

df_info = pd.merge(pd.merge(qual, assig_info), reg_info)
df_info_norm = pd.merge(pd.merge(qual_norm, assig_info), reg_info)

In [2]:
# Primer del grau d'Enginyeria Informàtica
# * PROGRAMACIÓ I                -> 364288
# * DISSENY DIGITAL BÀSIC        -> 364289
# * INTRODUCCIÓ ALS ORDINADORS   -> 364290
# * ÀLGEBRA                      -> 364291
# * CÀLCUL                       -> 364292
# * MATEMÀTICA DISCRETA          -> 364293
# * FÍSICA                       -> 364294
# * ALGORÍSMICA                  -> 364298
# * PROGRAMACIÓ II               -> 364299
# * ESTRUCTURA DE DADES          -> 364301

# Segon del grau d'Enginyeria Informàtica
# * ELECTRÒNICA                              -> 364297
# * ALGORÍSMICA AVANÇADA                     -> 364300
# * DISSENY DE SOFTWARE                      -> 364303
# * ESTRUCTURA DE COMPUTADORS                -> 364305
# * INTRODUCCIÓ A LA COMPUTACIÓ CIENTÍFICA   -> 364302
# * EMPRESA                                  -> 364296
# * PROBABILITATS I ESTADÍSTICA              -> 364295
# * PROGRAMACIÓ D'ARQUITECTURES ENCASTADES   -> 364306
# * PROJECTE INTEGRAT DE SOFTWARE            -> 364304
# * SISTEMES OPERATIUS I                     -> 364307

assigs1 = [364288, 364289, 364290, 364291, 364292, 364293, 364294, 364298, 364299, 364301]
assigs2 = [364297, 364300, 364303, 364305, 364302, 364296, 364295, 364306, 364304, 364307]
assigs = assigs1 + assigs2

In [3]:
recomender_table = table_students(df_info, assigs)
recomender_table_norm = table_students(df_info_norm, assigs)

primer = recomender_table[assigs1]
segon = recomender_table[assigs2]
primer_norm = recomender_table_norm[assigs1]
segon_norm = recomender_table_norm[assigs2]

In [4]:
def coefPearson(x, y):
    x = np.array(x)
    x = x.reshape(x.size)
    y = np.array(y)
    y = y.reshape(y.size)
    length = len(x)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    x_2 = y_2 = xy =0
    for i in range(length):#apliquem la fórmula del coeficient de pearson
        x_menys_mean = x[i]-x_mean
        y_menys_mean = y[i]-y_mean
        xy += (x_menys_mean)*(y_menys_mean)
        x_2 += x_menys_mean**2
        y_2 += y_menys_mean**2
    square = math.sqrt(x_2*y_2)
    if square == 0:#si square es 0 retornem 0 perquè no doni error
        return 0
    coef = float(xy)/square
    return abs(coef)

### Construcció d'un recomanador amb l'interfaç d'sklearn

Construïm un recomanador on el seu input es una matriu de NxM (alumne x assignatura, o viceversa). D'aquesta matriu agafa totes aquelles cel·les que tenen un NaN (np.nan) i prediu quina nota hauria d'anar, seguint la següent fórmula:

$$m_{e_qa_p} = \sum_{j=1}^n{\alpha_{a_pa_j}m_{e_qa_j}}$$

On:
 * $e_i$ és un estudiant
 * $a_i$ és una assignatura
 * $\alpha$ és una funció de similitud
 
Aquesta fórmula no és més que una suma ponderada, resumint, si un alumne s'assembla molt a un altre, tendiran a tenir les mateixes notes.

La aventatge que té aquest algoritme es que si transposem la matriu $m$, en comptes de tenir un recomanador col·laboratiu, tenim un recomanador basat en contingut.

A partir d'aquí hem de encapsular tot això dintre d'un estimador d'sklearn, per això hem d'heredar de `BaseEstimator` i implementar els mètodes `fit` i `predict`.

In [5]:
class Recomender(BaseEstimator):
    def __init__(self, method=coefPearson, collaborative=True):
        self._m = None
        self._method = method
        self._collaborative = collaborative
        
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self._m = np.concatenate((X,y), axis=1)
        return self
    
    def predict(self, X):
        X = np.array(X)
        if len(X.shape) == 1:
            X = np.array([X])
        nan = [np.nan for i in xrange(self._m.shape[1] - X.shape[1])]
        out = np.array([])
        for xi in X:
            m = np.array(self._m)
            row = np.concatenate((xi,nan),axis=1)
            m = np.vstack((m, row))
            if not self._collaborative:
                m = m.T
            m_predict, idx = self._recomender(m)
            if out.shape[0] == 0:
                out = np.array([[m_predict[tuple(i)] for i in idx]])
            else:
                out = np.vstack((out, [m_predict[tuple(i)] for i in idx]))
        return out
        
    def _recomender(self, m):
        out = np.array(m)
        idx = np.argwhere(np.isnan(m))
        for i in idx:
            out[tuple(i)] = 0
            sim_sum = 0
            cols = np.argwhere(~np.isnan(m[i[0]])).T[0]

            student = m[i[0]]
            for j in range(m.shape[0]):
                cols= list(set(cols).intersection(np.argwhere(~np.isnan(m[j])).T[0]))
                if np.isnan(m[j,i[1]]) or j == i[0]:
                    continue
                alpha = self._method(student[cols], m[j,cols])
                sim_sum += alpha
                out[tuple(i)] += alpha*m[j,i[1]]
            out[tuple(i)] /= sim_sum
        return out, idx
    

Ara provarem el nostre recomanador basat en contingut, és a dir, en assignatures.
Per començar provem de predir les notes d'un alumne amb un expedient excel·lent, d'aquest podrem veure que treu bones notes en tot.
Ara bé, que passa si un altre alumne treu males notes a les assignatures de programació?
* **'Programació I'**        - 0.5
* **'Algorítmica'**          - 3.4
* **'Programació II'**       - 1.2
* **'Estructura de Dades'**  - 0.7

In [6]:
to_predict = [[9.00, 9.00, 9.00, 9.00, 9.3, 9.00, 7.70, 9.10, 9.30, 7.40],
              [0.50, 9.00, 9.00, 9.00, 9.3, 9.00, 7.70, 3.4, 1.2, 0.7]]
recomender = Recomender(collaborative=False)
recomender.fit(primer.values, segon.values)
recomender.predict(to_predict)

array([[ 8.8509229 ,  8.87694108,  8.71400149,  8.98309551,  8.82963526,
         8.85978464,  8.86224037,  8.9216842 ,  8.65945579,  8.8854686 ],
       [ 6.5248692 ,  6.39545225,  3.76761985,  7.78963448,  7.04361254,
         6.37301966,  7.97467385,  6.25695891,  2.63530863,  6.86469523]])

Es veu com en les assignatures de segon que no estan molt relacionades amb les suspeses a primer no es veuen afectades, però tant 'Disseny de Software' i 'Projecte integrat de software' surten suspeses. Per tant, ja el mateix recomanador ens diu les **depèndencies d'assignatures**.

### Testeig

Defineixo diferents tècniques:
 * Recomanador col·laboratiu (alumnes)
 * Recomanador basat en contingut (assignatures)
 * Random Forest Regressor (regressor)
 * Linear Regression (regressor)
 
Per cada tècnica mesurem:
 * MAE (Error promig absolut): $$ \mathrm{MAE} = \frac{1}{n}\sum_{i=1}^n \left| y_{pred}-y_{test}\right|$$
 * MSE (Error promig quadràtic): $$\mathrm{MSE}=\frac{1}{n}\sum_{i=1}^n(y_{pred}-y_{test})^2$$
 * PCC (Coeficient de pearson): $$\mathrm{PCC} =\frac{\sum_{i=1}^n(y_{pred} - \bar{y}_{pred})(y_{test} - \bar{y}_{test})}{\sqrt{\sum_{i=1}^n(y_{pred} - \bar{y}_{pred})^2  \sum_{i=1}^n(y_{test} - \bar{y}_{test})^2}}$$
 

In [7]:
def benchmark(X,y,estimator):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    print "MAE (Mean Absolut Error):             %.3f" % mean_absolute_error(y_pred, y_test)
    print "MSE (Mean Squared Error):             %.3f" % mean_squared_error(y_pred, y_test)
    print "PCC (Pearson Correlation Coeficient): %.3f" % coefPearson(y_pred, y_test)

In [8]:
estimators = [Recomender(collaborative=True),
              Recomender(collaborative=False), 
              RandomForestRegressor(n_estimators=100, random_state=42), #42, magic number
              LinearRegression()]
print "No normalized data"
print "_________________________________________________________________"
print
for e  in estimators:
    print "Estimador:", e
    benchmark(primer.values, segon.values, e)
    print
print "Normalized data"
print "_________________________________________________________________"
print
for e  in estimators:
    print "Estimador:", e
    benchmark(primer_norm.values, segon_norm.values, e)
    print

No normalized data
_________________________________________________________________

Estimador: Recomender(collaborative=None, method=None)
MAE (Mean Absolut Error):             1.231
MSE (Mean Squared Error):             2.997
PCC (Pearson Correlation Coeficient): 0.335

Estimador: Recomender(collaborative=None, method=None)
MAE (Mean Absolut Error):             1.197
MSE (Mean Squared Error):             2.905
PCC (Pearson Correlation Coeficient): 0.403

Estimador: RandomForestRegressor(bootstrap=True, compute_importances=None,
           criterion='mse', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
           min_samples_split=2, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=42, verbose=0)
MAE (Mean Absolut Error):             1.134
MSE (Mean Squared Error):             2.584
PCC (Pearson Correlation Coeficient): 0.490

Estimador: LinearRegression(copy_X=True, fit_intercept=True, normalize=False)
M

### Ranking d'assignatures

A partir d'un bon recomanador o regressor, escollit segons el testeig, l'agafem per tal de fer un ranking d'assignatures. La finalitat és que donat un alumne poguem donar un ranking de notes ordenades des de la que li anirà bé fins la que tindrà més problemes.

In [9]:
def ranking_notes(X_train, y_train, X_test, estimator):
    estimator.fit(X_train, y_train)
    y_pred = np.squeeze(estimator.predict(X_test))
    s = sorted(zip(y_pred,assigs2))[::-1]
    return s

In [10]:
# Primer del grau d'Enginyeria Informàtica
# * PROGRAMACIÓ I                -> 364288
# * DISSENY DIGITAL BÀSIC        -> 364289
# * INTRODUCCIÓ ALS ORDINADORS   -> 364290
# * ÀLGEBRA                      -> 364291
# * CÀLCUL                       -> 364292
# * MATEMÀTICA DISCRETA          -> 364293
# * FÍSICA                       -> 364294
# * ALGORÍSMICA                  -> 364298
# * PROGRAMACIÓ II               -> 364299
# * ESTRUCTURA DE DADES          -> 364301

# Segon del grau d'Enginyeria Informàtica
# * ELECTRÒNICA                              -> 364297
# * ALGORÍSMICA AVANÇADA                     -> 364300
# * DISSENY DE SOFTWARE                      -> 364303
# * ESTRUCTURA DE COMPUTADORS                -> 364305
# * INTRODUCCIÓ A LA COMPUTACIÓ CIENTÍFICA   -> 364302
# * EMPRESA                                  -> 364296
# * PROBABILITATS I ESTADÍSTICA              -> 364295
# * PROGRAMACIÓ D'ARQUITECTURES ENCASTADES   -> 364306
# * PROJECTE INTEGRAT DE SOFTWARE            -> 364304
# * SISTEMES OPERATIUS I                     -> 364307

#input
alumne = [9.00, 9.00, 9.00, 1.3, 6.5, 2.2, 3.4, 9.10, 9.30, 7.40]
estimator = Recomender(collaborative=False)

# call function
s = ranking_notes(primer.values, segon.values, alumne, estimator)
for mark, subject in s:
    print "%.2f - %s" % (mark, assig[assig.id_assig == subject]['desc_assig'].values[0])

7.64 - Projecte Integrat de Software
7.44 - Programació d'Arquitectures Encastades
6.93 - Disseny de Software
6.64 - Algorísmica Avançada
6.61 - Empresa
6.40 - Sistemes Operatius I
6.09 - Introducció a la Computació Científica
6.01 - Probabilitats i Estadística
5.80 - Estructura de Computadors
5.73 - Electrònica
